In [4]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.ops import resources
from tensorflow.contrib.tensor_forest.python import tensor_forest

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from tensorflow.examples.tutorials.mnist import input_data
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False,
                                  source_url='http://yann.lecun.com/exdb/mnist/')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
num_steps = 500
batch_size = 1024
num_classes = 10
num_features = 784
num_trees = 10
max_nodes = 1000

X = tf.placeholder(tf.float32, shape = [None, num_features])
Y = tf.placeholder(tf.float32, shape = [None])

hyperparam = tensor_forest.ForestHParams(num_classes = num_classes, 
                                         num_features = num_features, 
                                         num_trees=num_trees, 
                                         max_nodes = max_nodes).fill()

In [4]:
#Build random forrest
forest_graph = tensor_forest.RandomForestGraphs(hyperparam)
tf.contrib.tensor_forest.client.random_forest.TensorForestEstimator(params)
# Training Graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

W0926 23:40:08.787492 23788 deprecation.py:506] From C:\Users\Zongyue Li\Documents\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [1]:
#Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.arg_max(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

NameError: name 'forest_graph' is not defined

In [6]:
#initialize all variables and forest resource
init_vars = tf.group(tf.global_variables_initializer(),
                    resources.initialize_resources(resources.shared_resources()))

In [7]:
#start Session
sess = tf.train.MonitoredSession()

sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    #Prepare Data and get the next batch of MNIST data
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict = {X : batch_x, Y : batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict = {X: batch_x, Y : batch_y})
        print("Step %i, Loss : %f, Acc : %f" %(i, l, acc))
        

W0926 23:40:45.138299 23788 deprecation.py:323] From C:\Users\Zongyue Li\Documents\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Step 1, Loss : -1.200000, Acc : 0.471680
Step 50, Loss : -252.600006, Acc : 0.885742
Step 100, Loss : -538.200012, Acc : 0.914062
Step 150, Loss : -832.000000, Acc : 0.940430
Step 200, Loss : -1001.000000, Acc : 0.924805
Step 250, Loss : -1001.000000, Acc : 0.929688
Step 300, Loss : -1001.000000, Acc : 0.927734
Step 350, Loss : -1001.000000, Acc : 0.934570
Step 400, Loss : -1001.000000, Acc : 0.928711
Step 450, Loss : -1001.000000, Acc : 0.926758
Step 500, Loss : -1001.000000, Acc : 0.921875


In [8]:
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy : ", sess.run(accuracy_op, feed_dict = {X : test_x, Y : test_y}))

Test Accuracy :  0.9219
